# Flotorch ADK - Agent with Tools Example

This notebook demonstrates how to create a Flotorch agent with custom tools.
The agent can use calculator, time, and text analysis tools.

## Requirements
- FLOTORCH_API_KEY environment variable
- FLOTORCH_BASE_URL environment variable


In [ ]:
%pip install --pre flotorch[adk]

In [ ]:
# Setup and imports
import asyncio
import os
import sys
from datetime import datetime
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

print("✓ Environment setup complete")


In [ ]:
# Flotorch ADK imports
from flotorch.adk.llm import FlotorchADKLLM
from google.adk.agents import LlmAgent
from google.adk.tools import FunctionTool
from google.adk.sessions import InMemorySessionService
from google.adk import Runner

print("✓ Flotorch ADK imports successful")


In [ ]:
# Configuration
APP_NAME = "flotorch_tools_example"
USER_ID = "flotorch_user_456"

print(f"App Name: {APP_NAME}")
print(f"User ID: {USER_ID}")


## Define Custom Tools

We'll create three useful tools:
1. **Calculator** - Safe math calculations
2. **Current Time** - Get current date and time
3. **Text Analyzer** - Analyze text statistics


In [ ]:
# Tool 1: Calculator
def calculator(expression: str) -> str:
    """Calculate basic math expressions safely.
    
    Args:
        expression: Math expression to calculate (e.g., '2 + 3 * 4')
        
    Returns:
        str: The calculation result or error message
    """
    try:
        allowed_chars = set('0123456789+-*/.() ')
        if not all(c in allowed_chars for c in expression):
            return "Error: Only basic math operations allowed"
        
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

# Test the calculator function
print("Testing calculator function:")
print(calculator("2 + 3 * 4"))
print(calculator("10 / 2"))


In [ ]:
# Tool 2: Current Time
def get_current_time() -> str:
    """Get the current date and time.
    
    Returns:
        str: Current date and time formatted as string
    """
    now = datetime.now()
    return f"Current time: {now.strftime('%Y-%m-%d %H:%M:%S')}"

# Test the time function
print("Testing time function:")
print(get_current_time())


In [ ]:
# Tool 3: Text Analyzer
def analyze_text(text: str) -> str:
    """Analyze text and provide statistics.
    
    Args:
        text: The text to analyze
        
    Returns:
        str: Text analysis with word count, character count, and other stats
    """
    word_count = len(text.split())
    char_count = len(text)
    char_count_no_spaces = len(text.replace(' ', ''))
    sentences = text.count('.') + text.count('!') + text.count('?')
    
    return f"""Text Analysis:
- Words: {word_count}
- Characters (with spaces): {char_count}
- Characters (without spaces): {char_count_no_spaces}
- Sentences: {sentences}"""

# Test the text analyzer function
print("Testing text analyzer function:")
print(analyze_text("Hello world! This is a test."))


In [ ]:
# Create Flotorch tools using FunctionTool
calculator_tool = FunctionTool(func=calculator)
time_tool = FunctionTool(func=get_current_time)
text_analyzer_tool = FunctionTool(func=analyze_text)

print("✓ Flotorch tools created:")
print("  - Calculator tool")
print("  - Time tool") 
print("  - Text analyzer tool")


In [ ]:
# Create Flotorch LLM model
model = FlotorchADKLLM(
    model_id="openai/gpt-4o-mini",
    api_key=os.getenv("FLOTORCH_API_KEY"),
    base_url=os.getenv("FLOTORCH_BASE_URL"),
)

print("✓ Flotorch LLM model created")


In [ ]:
# Create Flotorch agent with tools
agent = LlmAgent(
    name="Flotorch_Tool_Assistant",
    description="A helpful Flotorch AI assistant with calculator, time, and text analysis tools",
    instruction="You are Flotorch AI assistant with special tools. You can help with math calculations, tell current time, and analyze text.",
    model=model,
    tools=[calculator_tool, time_tool, text_analyzer_tool],
)

print("✓ Flotorch agent with tools created")


In [ ]:
# Setup runner
session_service = InMemorySessionService()
runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service
)

print("✓ Runner setup complete")


In [ ]:
# Helper function to send messages
def run_single_turn(query, session_id, user_id):
    """Send message to Flotorch agent and get response."""
    from google.genai import types
    
    content = types.Content(role="user", parts=[types.Part(text=query)])
    events = runner.run(user_id=user_id, session_id=session_id, new_message=content)

    for event in events:
        if event.is_final_response():
            if event.content and event.content.parts:
                return event.content.parts[0].text
    
    return "Sorry, I couldn't process that."

print("✓ Helper function defined")


In [ ]:
# Create session and test the agent with tools
async def test_agent_with_tools():
    session = await runner.session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
    )
    
    print(f"Flotorch agent with tools ready! Session: {session.id}")
    print("Available tools: Calculator, Current Time, Text Analyzer")
    
    # Test calculator tool
    test_message = "What's 15 * 7?"
    response = run_single_turn(test_message, session.id, USER_ID)
    print(f"\nUser: {test_message}")
    print(f"Flotorch AI: {response}")
    
    return session

# Run the test
session = await test_agent_with_tools()


In [ ]:
# Interactive chat function with tools
def chat_with_tools(message):
    """Simple function to chat with the agent using tools."""
    response = run_single_turn(message, session.id, USER_ID)
    print(f"You: {message}")
    print(f"Flotorch AI: {response}")
    return response

# Try the different tools
print("=== Testing Calculator Tool ===")
chat_with_tools("Calculate 25 + 17 * 3")

print("\n" + "="*50)
print("=== Testing Time Tool ===")
chat_with_tools("What time is it?")

print("\n" + "="*50)
print("=== Testing Text Analyzer Tool ===")
chat_with_tools("Analyze this text: Flotorch makes AI development easy!")


## Summary

This notebook demonstrated:
- ✓ Creating custom tools (calculator, time, text analyzer)
- ✓ Using FunctionTool to wrap Python functions
- ✓ Building an agent with multiple tools
- ✓ Testing each tool through the agent

You can now use `chat_with_tools("your message")` to interact with the agent and its tools!
